<a href="https://colab.research.google.com/github/24-FYP-Automated-Feedback-Generation/Adaptive-attention/blob/main/cross_attention_with_prompts_and_weights_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uom200407h_modified_dataset_path = kagglehub.dataset_download('uom200407h/modified-dataset')
uom200644f_metacognitive_dataset_path = kagglehub.dataset_download('uom200644f/metacognitive-dataset')
bashadithennakoon_metacognitive_feedback_for_algorithm_solving_path = kagglehub.dataset_download('bashadithennakoon/metacognitive-feedback-for-algorithm-solving')

print('Data source import complete.')


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uom200407h_modified_dataset_path = kagglehub.dataset_download('uom200407h/modified-dataset')
uom200644f_metacognitive_dataset_path = kagglehub.dataset_download('uom200644f/metacognitive-dataset')
bashadithennakoon_metacognitive_feedback_for_algorithm_solving_path = kagglehub.dataset_download('bashadithennakoon/metacognitive-feedback-for-algorithm-solving')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/metacognitive-dataset/metacognitive-dataset.csv
/kaggle/input/modified-dataset/modified_dataset.csv
/kaggle/input/metacognitive-feedback-for-algorithm-solving/final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini.csv


In [ ]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer, GPT2Model,GPT2Tokenizer,GPT2LMHeadModel
from transformers import AutoTokenizer, T5ForConditionalGeneration , T5Tokenizer , T5Model
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
checkpoint = "t5-base"

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
t5_tokenizer.vocab_size

32100

In [ ]:
#set the max length to model's default present max length
t5_tokenizer.model_max_length = t5_tokenizer.model_max_length

In [ ]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [ ]:
file_path = "/kaggle/input/metacognitive-feedback-for-algorithm-solving/final_dataset_with_annotated_metacognitive_feedback_gpt-4o-mini.csv"
df = pd.read_csv(file_path)

In [ ]:
df.columns

Index(['Question 1', 'Response 1', 'Right answer 1', 'Q01', 'Q02', 'Q03',
       'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13',
       'Q14', 'Q15', 'Q16', 'metacognitive_vector', 'metacognitive_feedback'],
      dtype='object')

In [ ]:
df.rename(
    columns={
        'Question 1': 'Problem',
        'Response 1': 'Student_code',
        'Right answer 1': 'Expected_code'
    },
    inplace=True
)

In [ ]:
df.head(3)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback
0,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...","Your initial code serves as a starting point, ..."
1,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your code exhibits a solid attempt at reading ...
2,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,3 : Often,3 : Often,2 : Sometimes,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",It looks like you're in a good place with some...


In [ ]:
student_prompt = "This is the combination of the problem desccription and the student provided code."
context_prompt = "This is the combination of the problem description and the expected correct answer for that algorithm design question."
metacognitive_components = [
                "Read", "Identify", "Rephrase", "Examples", "Breakdown", "Estimate", "Plan",
                "Revise", "Verify", "AvoidMistakes", "MonitorSteps", "MonitorProcess",
                "ValidateConstraints", "Confirm", "CheckRequirements", "Reflect"
            ]

metacognition_prompt = (
            f"Metacognitive feedback helps students reflect on their algorithm-solving strategies. "
            f"The passed metacognitive vector is a 16-dimensional vector describing the student's metacognition profile:  "
            f"where each dimension represents one of 16 metacognitive components, rated as 1 (Almost never), "
            f"2 (Sometimes), or 3 (Often). These components include: {', '.join(metacognitive_components)}."
        )
decoder_prompt = "This is the combination student's metacognition profile and the student's code for the algorithm problem with the problem description itself."


In [ ]:
df['combined_problem_student'] = student_prompt + "  Problem:  " + df['Problem'] + ".  Student provided code:  " + df['Student_code']

In [ ]:
df['combined_problem_expected'] = context_prompt + " Problem:  " + df['Problem'] + ".  Expected correct ansswer for the problem:  " + df['Expected_code']

In [ ]:
df['combined_metacogntion_prompt'] =  metacognition_prompt + "Here is the student's metacognition vector: " + df['metacognitive_vector']

In [ ]:
# df['combined_persona_student'] =

In [ ]:
df['combined_metacogntion_prompt'][20]

"Metacognitive feedback helps students reflect on their algorithm-solving strategies. The passed metacognitive vector is a 16-dimensional vector describing the student's metacognition profile:  where each dimension represents one of 16 metacognitive components, rated as 1 (Almost never), 2 (Sometimes), or 3 (Often). These components include: Read, Identify, Rephrase, Examples, Breakdown, Estimate, Plan, Revise, Verify, AvoidMistakes, MonitorSteps, MonitorProcess, ValidateConstraints, Confirm, CheckRequirements, Reflect.Here is the student's metacognition vector: ['2 ', '3 ', '2 ', '3 ', '2 ', '2 ', '1 ', '2 ', '3 ', '3 ', '3 ', '3 ', '3 ', '3 ', '2 ', '2 ']"

In [ ]:
df.columns

Index(['Problem', 'Student_code', 'Expected_code', 'Q01', 'Q02', 'Q03', 'Q04',
       'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14',
       'Q15', 'Q16', 'metacognitive_vector', 'metacognitive_feedback',
       'combined_problem_student', 'combined_problem_expected',
       'combined_metacogntion_prompt'],
      dtype='object')

In [ ]:
df.dropna(subset=['Problem', 'metacognitive_feedback', 'combined_problem_student','Student_code'], inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isnull().sum()

Problem                         0
Student_code                    0
Expected_code                   0
Q01                             0
Q02                             0
Q03                             0
Q04                             0
Q05                             0
Q06                             0
Q07                             0
Q08                             0
Q09                             0
Q10                             0
Q11                             0
Q12                             0
Q13                             0
Q14                             0
Q15                             0
Q16                             0
metacognitive_vector            0
metacognitive_feedback          0
combined_problem_student        0
combined_problem_expected       0
combined_metacogntion_prompt    0
dtype: int64

In [ ]:
df['metacognitive_feedback'][100]

"Your current implementation shows a good effort in structuring your code with functions, but there are some areas where further refinement is necessary to meet the problem requirements. First, consider how you're capturing the relationships between birth years and heights. While you are correctly gathering names, birthdates, and heights into a dictionary, think about how you can aggregate heights by decade instead of year. This will streamline the calculation of average heights. It’s crucial to loop through your input list once to comprehend and process the data before beginning any calculations for averaging—possibly consolidating this logic in your `calculate_average_height` function. Also, ensure that you are converting heights to the correct data type before performing any arithmetic operations. Additionally, take a closer look at how you’re determining the range of decades; right now it seems like you may be focusing on unique years instead of decades. Consider creating a systema

In [ ]:
df.head(5)

,Problem,Student_code,Expected_code,Q01,Q02,Q03,Q04,Q05,Q06,Q07,...,Q12,Q13,Q14,Q15,Q16,metacognitive_vector,metacognitive_feedback,combined_problem_student,combined_problem_expected,combined_metacogntion_prompt
0,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...","Your initial code serves as a starting point, ...",This is the combination of the problem desccri...,This is the combination of the problem descrip...,Metacognitive feedback helps students reflect ...
1,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your code exhibits a solid attempt at reading ...,This is the combination of the problem desccri...,This is the combination of the problem descrip...,Metacognitive feedback helps students reflect ...
2,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",It looks like you're in a good place with some...,This is the combination of the problem desccri...,This is the combination of the problem descrip...,Metacognitive feedback helps students reflect ...
3,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your approach to reading the file and splittin...,This is the combination of the problem desccri...,This is the combination of the problem descrip...,Metacognitive feedback helps students reflect ...
4,Develop a Python program that takes the name o...,file_input = input() file_open = open(fil...,"def substitute_vowels(chunk, vowel_substitutes...",3 : Often,2 : Sometimes,1 : Almost Never,3 : Often,2 : Sometimes,2 : Sometimes,1 : Almost Never,...,2 : Sometimes,2 : Sometimes,3 : Often,3 : Often,1 : Almost Never,"['3 ', '2 ', '1 ', '3 ', '2 ', '2 ', '1 ', '3 ...",Your initial approach to the problem is a good...,This is the combination of the problem desccri...,This is the combination of the problem descrip...,Metacognitive feedback helps students reflect ...


In [ ]:
from torch.utils.data import Dataset
import ast
class CustomDataset(Dataset):
    def __init__(self, dataset, t5_tokenizer,gpt2_tokenizer, max_length=512):
        self.t5_tokenizer = t5_tokenizer
        self.gpt2_tokenizer = gpt2_tokenizer
        self.data = dataset
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        metacognitive_prompt = self.data['combined_metacogntion_prompt'][idx]
        problem_student_code = self.data['combined_problem_student'][idx]
        problem_expected_code = self.data['combined_problem_expected'][idx]
        student_code = self.data['Student_code'][idx]
        target = self.data['metacognitive_feedback'][idx]

        # metacognitive_vector_float = [
        # float(item.strip()) for item in ast.literal_eval(metacognitive_vector)]
        # metacognition_vector_ids = torch.tensor(metacognitive_vector_float, dtype=torch.float)
        metacognition_prompt_ids = torch.tensor(
            self.t5_tokenizer.encode(metacognitive_prompt, max_length=self.max_length, truncation=True, padding="max_length")
        )

        problem_student_code_ids = torch.tensor(
            self.t5_tokenizer.encode(problem_student_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        problem_expected_code_ids = torch.tensor(
            self.t5_tokenizer.encode(problem_expected_code, max_length=self.max_length, truncation=True, padding="max_length")
        )

        student_code_ids = torch.tensor(
            self.t5_tokenizer.encode(student_code, max_length=self.max_length, truncation=True, padding="max_length")
        )
        target_ids = torch.tensor(
            self.t5_tokenizer.encode(target, max_length=self.max_length, truncation=True, padding="max_length")
        )

        return metacognition_prompt_ids, problem_student_code_ids, problem_expected_code_ids, student_code_ids, target_ids

In [ ]:
dataset = CustomDataset(df, t5_tokenizer, gpt2_tokenizer)

In [ ]:
len(dataset)

16803

In [ ]:
metacognition_prompt_ids, problem_student_code_ids, problem_expected_code_ids, student_code_ids, target_ids = dataset[4000]
print(f"Metacognition vector IDs: {metacognition_prompt_ids}")
print(f"Expected feedback IDs: {problem_student_code_ids}")
print(f"Expected encoded feedback IDs: {problem_expected_code_ids}")
print(f"Student Answer IDs: {student_code_ids.shape}")
print(f"Target IDs: {target_ids.shape}")
print("\n")

Metacognition vector IDs: tensor([14204,    75, 12905,  3268,  3160,  1691,   481,  3548,    30,    70,
        12628,    18,  6065,    53,  3266,     5,    37,  2804, 10531,    75,
        12905,  3268, 12938,    19,     3,     9,   898,    18, 11619, 12938,
            3, 16012,     8,  1236,    31,     7, 10531,    75, 12905,  1575,
         3278,    10,   213,   284,  9340,  5475,    80,    13,   898, 10531,
           75, 12905,  3268,  3379,     6,     3,  4094,    38,   209,    41,
        13283,   470,   201,   204,    41, 19055,   715,     7,   201,    42,
          220,    41, 10084,   137,   506,  3379,   560,    10,  3403,     6,
            3, 23393,     6,   419, 27111,     6, 19119,     6, 11429,  3035,
            6, 23621,    15,     6,  2926,     6,  6342,   159,    15,     6,
          781,  4921,     6, 15856,   329,   159,  4914,     7,     6, 15192,
        14337,   102,     7,     6, 15192,  3174,  2319,     7,     6, 23545,
          342,  4302,     7,  9719,   

In [ ]:
gpt2_tokenizer.pad_token

'<|endoftext|>'

In [ ]:
gpt2_pad_token_id = gpt2_tokenizer.pad_token_id

In [ ]:
gpt2_pad_token_id

50256

In [ ]:
t5_tokenizer.pad_token

'<pad>'

In [ ]:
t5_pad_token_id = t5_tokenizer.pad_token_id

In [ ]:
t5_pad_token_id

0

# Context Enocder

In [ ]:
# class ContextEncoder(nn.Module):
#     def __init__(self, t5_model_name='t5-base', output_dim=768 , max_length = 512):
#         super(ContextEncoder, self).__init__()

#         self.max_length = max_length

#         self.t5_encoder = T5Model.from_pretrained(t5_model_name).encoder
#         self.t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
#         self.fc = nn.Linear(self.t5_encoder.config.d_model, output_dim)

#     def forward(self, problem_code_ids, attention_masks=None, prompt=""):

#         prompt_ids = torch.tensor(
#             self.t5_tokenizer.encode(prompt, max_length=self.max_length, truncation=True, padding="max_length")
#         )

#         problem_code = [self.t5_tokenizer.decode(ids, skip_special_tokens=True) for ids in problem_code_ids]

#         combined_text = prompt + " " + " ".join(problem_code)

#         encoded = self.t5_tokenizer(
#             combined_text,
#             max_length=self.max_length,
#             truncation=True,
#             padding="max_length",
#             return_tensors="pt"
#         ).to(problem_code_ids.device)

#         encoder_outputs = self.t5_encoder(
#             input_ids=encoded["input_ids"],
#             attention_mask=encoded["attention_mask"]
#         )

#         context_hidden_states = encoder_outputs.last_hidden_state

#         context_rep = context_hidden_states.mean(dim=1)

#         # decoded_combined = [
#         # self.t5_tokenizer.decode(ids, skip_special_tokens=True)
#         # for ids in encoded_outputs]

#         context_rep = self.fc(context_rep)
#         final_rep = context_rep.unsqueeze(1)

#         return final_rep

In [ ]:
# context_encoder = ContextEncoder().to(device)

In [ ]:
# context_encoder

In [ ]:

# class MetacognitionLayer(nn.Module):
#     def __init__(self, metacognitive_dim=16, output_dim=768):
#         super(MetacognitionLayer, self).__init__()
#         #16 to 768 mapping
#         self.metacognitive_fc = nn.Linear(metacognitive_dim, output_dim)
#         self.final_fc = nn.Linear(output_dim, output_dim)

#         self.tokenizer = T5Tokenizer.from_pretrained("t5-base")
#         self.t5_pad_token_id = self.tokenizer.pad_token_id
#         self.encoder = T5Model.from_pretrained("t5-base").encoder



#         for param in self.encoder.parameters():
#             param.requires_grad = False

#     def forward(self, metacognitive_vector):

#         metacognitive_components = [
#                 "Read", "Identify", "Rephrase", "Examples", "Breakdown", "Estimate", "Plan",
#                 "Revise", "Verify", "AvoidMistakes", "MonitorSteps", "MonitorProcess",
#                 "ValidateConstraints", "Confirm", "CheckRequirements", "Reflect"
#             ]

#         prompt_text = (
#             f"Metacognitive feedback helps students reflect on their algorithm-solving strategies. "
#             f"The passed metacognitive vector is a 16-dimensional vector describing the student's metacognition profile, "
#             f"where each dimension represents one of 16 metacognitive components, rated as 1 (Almost never), "
#             f"2 (Sometimes), or 3 (Often). These components include: {', '.join(metacognitive_components)}."
#         )


#         input_prompt = self.tokenizer(
#         prompt_text,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=512-16
#         )
#         input_prompt = {key: value.to(metacognitive_vector.device) for key, value in input_prompt.items()}

#         outputs = self.encoder(input_ids=input_prompt["input_ids"], attention_mask=input_prompt["attention_mask"])
#         prompt_embedding = outputs.last_hidden_state.mean(dim=1)
#         prompt_embedding = prompt_embedding.to(metacognitive_vector.device)

#         metacognitive_rep = self.metacognitive_fc(metacognitive_vector)
#         final_rep = self.final_fc(metacognitive_rep)
#         persona_rep = final_rep.unsqueeze(1)

#         final_persona = persona_rep + prompt_embedding

#         return final_persona

In [ ]:
# metacognitive_emb = MetacognitionLayer().to(device)

In [ ]:
# metacognitive_emb

In [ ]:

class PAALayer(nn.Module):
    def __init__(self, hidden_dimension = 512 , tau=0.8,dropout_rate=0.1):
        super(PAALayer, self).__init__()
        self.hidden_dimenstion = hidden_dimension
        self.tau = tau


        self.fc = nn.Linear(2 * hidden_dimension, hidden_dimension)
        self.sigmoid = nn.Sigmoid()
        self.fc_out = nn.Linear(hidden_dimension, hidden_dimension)
        self.dropout = nn.Dropout(p=dropout_rate)


    def forward(self, hR , oP, oC):

        Mp_input  = torch.cat([hR,oP], dim=-1)
        Mp = self.fc(Mp_input)
        Wp = self.sigmoid(Mp)

        Mpersona = Wp
        Mcontext = 1 - Wp

        oP_weighted = Mcontext * oP
        oC_weighted = Mpersona * oC

        HPAA = oP_weighted + oC_weighted

        output = self.fc_out(HPAA)

        return output

In [ ]:
paa = PAALayer()

In [ ]:
paa

PAALayer(
  (fc): Linear(in_features=1024, out_features=512, bias=True)
  (sigmoid): Sigmoid()
  (fc_out): Linear(in_features=512, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
class CustomTransformerBlock(nn.Module):
    def __init__(self, hidden_size, tau , dropout_rate=0.1):
        super(CustomTransformerBlock, self).__init__()

        self.input_self_attention = nn.MultiheadAttention(hidden_size, num_heads=8, batch_first=True)
        self.context_attn = nn.MultiheadAttention(hidden_size, num_heads=8, batch_first=True)
        self.persona_attn = nn.MultiheadAttention(hidden_size, num_heads=8, batch_first=True)
        self.persona_proj = nn.Linear(768, hidden_size)
        self.context_proj = nn.Linear(768, hidden_size)

        self.paa_layer = PAALayer(hidden_dimension=hidden_size, tau=tau)


        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(2048, hidden_size),
            nn.LayerNorm(hidden_size)
        )
        self.layer_norm2 = nn.LayerNorm(hidden_size)

    def forward(self, student_initial_state, encoded_persona, encoded_context):

        hR, _ = self.input_self_attention(student_initial_state, student_initial_state, student_initial_state) #query
        encoded_persona = self.persona_proj(encoded_persona)
        encoded_context = self.context_proj(encoded_context)
        # print("hr shape",hR.shape)
        # print("encoded persona shape", encoded_persona.shape)
        # print("encoded context shape", encoded_context.shape)
        oP, _ = self.persona_attn(hR, encoded_persona, encoded_persona )

        # encoded_context = encoded_context.repeat(hR.size(0), hR.size(1), 1)

        oC, _ = self.context_attn(hR, encoded_context, encoded_context )

        HPAA = self.paa_layer(hR, oP, oC)

        mlp_output = self.mlp(HPAA)
        output = self.layer_norm2(mlp_output)

        return output


In [ ]:
custom_layer = CustomTransformerBlock(768,0.5)

In [ ]:
custom_layer

CustomTransformerBlock(
  (input_self_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (context_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (persona_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (persona_proj): Linear(in_features=768, out_features=768, bias=True)
  (context_proj): Linear(in_features=768, out_features=768, bias=True)
  (paa_layer): PAALayer(
    (fc): Linear(in_features=1536, out_features=768, bias=True)
    (sigmoid): Sigmoid()
    (fc_out): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (mlp): Sequential(
    (0): Linear(in_features=768, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=2048, out_features=76

In [ ]:
class PAAModel(nn.Module):
    def __init__(self, hidden_size=512, vocab_size = 32100 ,tau=0.5, max_length=512, num_transformer_blocks=4):
        super(PAAModel , self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.num_transformer_blocks = num_transformer_blocks

        self.t5_encoder = T5Model.from_pretrained("t5-base").encoder
        for param in self.t5_encoder.parameters():
            param.requires_grad = False


        self.token_embedding = nn.Embedding(vocab_size, hidden_size)
        self.position_embedding = nn.Embedding(max_length, hidden_size)
        self.dropout = nn.Dropout(p=0.1)

        self.transformer_blocks = nn.ModuleList([CustomTransformerBlock(hidden_size, tau) for _ in range(num_transformer_blocks)])
        self.final_fc = nn.Linear(hidden_size, vocab_size)
        self.layer_norm = nn.LayerNorm(hidden_size)

    def forward(self,  metacognition_prompt_ids,
                       problem_student_code_ids ,
                       problem_expected_code_ids,
                       metacognition_attention_mask,
                       expected_attention_mask):

        with torch.no_grad():

            metacognition_prompt_encoded = self.t5_encoder(
                                                        input_ids=metacognition_prompt_ids,
                                                        attention_mask=metacognition_attention_mask).last_hidden_state

            problem_expected_code_encoded = self.t5_encoder(
                                                        input_ids=problem_expected_code_ids,
                                                        attention_mask=expected_attention_mask).last_hidden_state


        token_embeds = self.token_embedding(problem_student_code_ids)
        position_ids = torch.arange(0, 1, device=problem_student_code_ids.device).unsqueeze(0)
        position_ids = position_ids % self.max_length
        position_embeds = self.position_embedding(position_ids)

        inputs_embeds = token_embeds + position_embeds
        inputs_embeds = self.dropout(inputs_embeds)

        student_initial_state = inputs_embeds
        transformer_output = student_initial_state

        for transformer_block in self.transformer_blocks:
            transformer_output = transformer_block(transformer_output, metacognition_prompt_encoded, problem_expected_code_encoded)

        logits = self.final_fc(transformer_output)

        return logits

In [ ]:
model = PAAModel()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
model.to(device)

PAAModel(
  (t5_encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
              (act): R

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
LOSS = torch.nn.CrossEntropyLoss()

In [ ]:
num_epochs = 200

In [ ]:
df_train = df[0:5000]

In [ ]:
len(df_train)

5000

In [ ]:
df_train.isnull().sum()

Problem                         0
Student_code                    0
Expected_code                   0
Q01                             0
Q02                             0
Q03                             0
Q04                             0
Q05                             0
Q06                             0
Q07                             0
Q08                             0
Q09                             0
Q10                             0
Q11                             0
Q12                             0
Q13                             0
Q14                             0
Q15                             0
Q16                             0
metacognitive_vector            0
metacognitive_feedback          0
combined_problem_student        0
combined_problem_expected       0
combined_metacogntion_prompt    0
dtype: int64

In [ ]:
from torch.utils.data import DataLoader
train_dataset = CustomDataset(df_train, t5_tokenizer , gpt2_tokenizer)
train_dataloader = DataLoader(train_dataset , batch_size = 4 ,shuffle = True )

In [ ]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

In [ ]:
import os

In [ ]:
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
# t5_encoder = T5Model.from_pretrained(checkpoint).encoder

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# t5_encoder.to(device)

T5Stack(
  (embed_tokens): Embedding(32128, 768)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768, bias=False)
            (relative_attention_bias): Embedding(32, 12)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerFF(
          (DenseReluDense): T5DenseActDense(
            (wi): Linear(in_features=768, out_features=3072, bias=False)
            (wo): Linear(in_features=3072, out_features=768, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
            (act): ReLU()
          )
          (layer_norm): T5LayerNorm()
          (d

In [ ]:
for epoch in range(num_epochs):
    print(f"Training started for epoch {epoch + 1}/{num_epochs}")
    model.train()
    total_loss = 0

    for idx, (metacognition_prompt_ids,
              problem_student_code_ids,
              problem_expected_code_ids,
              student_code_ids,
              target_ids) in enumerate(train_dataloader):


        metacognition_prompt_ids = metacognition_prompt_ids.to(device)
        problem_student_code_ids = problem_student_code_ids.to(device)
        problem_expected_code_ids = problem_expected_code_ids.to(device)
        student_code_ids = student_code_ids.to(device)
        target_ids = target_ids.to(device)

        #attention masking
        student_attention_mask = (problem_student_code_ids != t5_pad_token_id).long().to(device)
        expected_attention_mask = (problem_expected_code_ids != t5_pad_token_id).long().to(device)
        metacognition_attention_mask = (metacognition_prompt_ids != t5_pad_token_id).long().to(device)

        #encoding the decoder inputs for cross attention
        # metacognition_prompt_encoded = t5_encoder(
        #                                             input_ids=metacognition_prompt_ids,
        #                                             attention_mask=metacognition_attention_mask).last_hidden_state

        # problem_expected_code_encoded = t5_encoder(
        #                                             input_ids=problem_expected_code_ids,
        #                                             attention_mask=expected_attention_mask).last_hidden_state



        optimizer.zero_grad()
        logits = model(metacognition_prompt_ids,
                       problem_student_code_ids ,
                       problem_expected_code_ids,
                      metacognition_attention_mask,
                      expected_attention_mask)


        logits = logits.view(-1, logits.size(-1))

        target_ids = target_ids.view(-1)
        # print(logits.shape)
        # print(target_ids.shape)


        loss = LOSS(logits, target_ids)
        total_loss += loss.item()


        loss.backward()
        # for name, param in model.named_parameters():
        #     if 'context_encoder' in name:
        #         assert param.grad is None, f"Gradients found in frozen encoder {name}"
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        if idx % 10 == 0:
            print(f"Batch {idx + 1}/{len(train_dataloader)} | Loss: {loss.item():.4f}" , end='\r')






    if epoch % 5 ==0 :
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    print(f"Layer: {name} | Grad Norm: {param.grad.norm().item()}")
                elif param.requires_grad:
                    print(f"Layer: {name} | Grad: None")

    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch + 1}.pth")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / max(len(train_dataloader), 1),
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")


    avg_loss = total_loss / max(len(train_dataloader), 1)
    #writer.add_scalar("Loss/train", avg_loss, epoch + 1)
    print(f"Epoch [{epoch + 1}/{num_epochs}] completed | Average Loss: {avg_loss:.4f}")

Training started for epoch 1/200
Layer: token_embedding.weight | Grad Norm: 3.9925669107626627e-13
Layer: position_embedding.weight | Grad Norm: 2.451501241590437e-12
Layer: transformer_blocks.0.input_self_attention.in_proj_weight | Grad Norm: 7.675216712188515e-11
Layer: transformer_blocks.0.input_self_attention.in_proj_bias | Grad Norm: 3.4575411110571164e-12
Layer: transformer_blocks.0.input_self_attention.out_proj.weight | Grad Norm: 9.322762700714193e-11
Layer: transformer_blocks.0.input_self_attention.out_proj.bias | Grad Norm: 6.135561243097998e-12
Layer: transformer_blocks.0.context_attn.in_proj_weight | Grad Norm: 6.447607447945813e-10
Layer: transformer_blocks.0.context_attn.in_proj_bias | Grad Norm: 4.4577869173778595e-10
Layer: transformer_blocks.0.context_attn.out_proj.weight | Grad Norm: 8.173368093444822e-10
Layer: transformer_blocks.0.context_attn.out_proj.bias | Grad Norm: 7.921868716564973e-10
Layer: transformer_blocks.0.persona_attn.in_proj_weight | Grad Norm: 7.6209

In [ ]:
torch.cuda.memory_summary(device='cuda', abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   7929 MiB |   7972 MiB |  14364 MiB |   6434 MiB |\n|       from large pool |   7927 MiB |   7970 MiB |  14332 MiB |   6405 MiB |\n|       from small pool |      1 MiB |      3 MiB |     31 MiB |     29 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   7929 MiB |   7972 MiB |  14364 MiB |   6434 MiB |\n|       from large pool |   7927 MiB |   7970 MiB |

In [ ]:
export CUDA_LAUNCH_BLOCKING=1

In [ ]:
torch.save(model.state_dict(), 'paamodel.pth')

In [ ]:
model = PAAModel()

# Load the saved state dict
model.load_state_dict(torch.load('paamodel.pth'))

In [ ]:
df_eval = df[9999:10000].reset_index(drop=True)

In [ ]:
df_eval

In [ ]:
eval_dataset = CustomDataset(df_eval, t5_tokenizer,gpt2_tokenizer)

In [ ]:


def inference(model,gpt2_tokenizer, t5_tokenizer, eval_dataset, device):
    model.eval()
    model.to(device)

    metacognitive_vector_ids, problem_student_code_ids, problem_expected_code_ids,student_code_ids, target_ids = eval_dataset[0]

    metacognitive_tensor = metacognitive_vector_ids.unsqueeze(0).to(device)
    problem_student_code_tensor = problem_student_code_ids.unsqueeze(0).to(device)
    problem_expected_code_tensor = problem_expected_code_ids.unsqueeze(0).to(device)
    target_tensor = target_ids.unsqueeze(0).to(device)

    student_attention_mask = (problem_student_code_tensor != t5_tokenizer.pad_token_id).long().to(device)
    expected_attention_mask = (problem_expected_code_tensor != t5_tokenizer.pad_token_id).long().to(device)


    with torch.no_grad():

        logits = model(
            metacognitive_vector_ids=metacognitive_tensor,
            problem_student_code_ids=problem_student_code_tensor,
            problem_expected_code_ids=problem_expected_code_tensor,
            expected_attention_mask=expected_attention_mask,
            student_attention_mask=student_attention_mask
        )

        predictions = logits.argmax(dim=-1).squeeze().tolist()
        filtered_tokens = [token for token in predictions if token != 0]
        #decoded_text = t5_tokenizer.decode(filtered_tokens, skip_special_tokens=False)
        decoded_text = t5_tokenizer.decode(predictions, skip_special_tokens=True)


        return filtered_tokens, decoded_text



In [ ]:
predictions, decoded_text = inference(model, gpt2_tokenizer, t5_tokenizer, eval_dataset, device)

In [ ]:
print("Predicted Tokens:", predictions)

In [ ]:
print("Decoded Text:", decoded_text)

In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer=None):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded: Epoch {epoch}, Loss: {loss:.4f}")
    return model, optimizer, epoch, loss

In [ ]:
checkpoint_path = "./checkpoints/model_epoch_20.pth"
model, optimizer, start_epoch, _ = load_checkpoint(checkpoint_path, model, optimizer)

In [ ]:
checkpoint_path = "./checkpoints/model_epoch_10.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))  # Use GPU if available: 'cuda'
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
len(df)

In [ ]:
df_eval1 = df[449:450].reset_index(drop=True)

In [ ]:
eval_dataset1 = CustomDataset(df_eval1, t5_tokenizer,gpt2_tokenizer)

In [ ]:
predictions, decoded_text = inference(model, gpt2_tokenizer, t5_tokenizer, eval_dataset1, device)

In [ ]:
print("Decoded Text:", decoded_text)